In [1]:
## question, what wikipedia version? I used the latest version, needs to confirm
from datasets import load_dataset
import pandas as pd
import json
import urllib 


## Load wikipedia
Load the latest wikipedia dump from huggingface

https://huggingface.co/datasets/wikipedia


In [209]:
wikidata_all = load_dataset("wikipedia", "20220301.en")

Found cached dataset wikipedia (/home/parham/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'wikimania2016wiki' is not defined

In [11]:
df_wiki_all_2022 = wikidata_all['train'].to_pandas()

In [12]:
df_wiki_all_2022

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,25,https://en.wikipedia.org/wiki/Autism,Autism,Autism is a neurodevelopmental disorder charac...
2,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the measure of the diffuse refl...
3,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
4,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
...,...,...,...,...
6458665,70201819,https://en.wikipedia.org/wiki/Bianca%20Fernandez,Bianca Fernandez,Bianca Jolie Fernandez (born 24 February 2004)...
6458666,70201882,https://en.wikipedia.org/wiki/Condons%20and%20...,Condons and Clangibbon,Condons and Clangibbon () is a barony in Count...
6458667,70201886,https://en.wikipedia.org/wiki/2022%20Chattanoo...,2022 Chattanooga Red Wolves SC season,The 2022 Chattanooga Red Wolves SC season will...
6458668,70201947,https://en.wikipedia.org/wiki/Nkiko%20Prosper,Nkiko Prosper,Turatsinze Nkiko Prosper (born 1985) professio...


## Parse into the right format for Anserini

In [43]:
df_wiki_all_2022_for_lucine= df_wiki_all_2022[['id','text']]
df_wiki_all_2022_for_lucine.rename(columns={'id':'id','text':'contents'},inplace=True)
df_wiki_all_2022_for_lucine.to_json('../data/input/wiki2022.jsonl',orient='records', lines=True)


## Index WikiPedia using for anserini

In [51]:
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input ../data/input \
  --index ../data/index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw \
  --bm25.accurate

2023-01-06 10:27:00,174 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-01-06 10:27:00,176 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-01-06 10:27:00,176 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-01-06 10:27:00,176 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: ../data/input
2023-01-06 10:27:00,176 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-01-06 10:27:00,177 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-01-06 10:27:00,177 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-01-06 10:27:00,177 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: en
2023-01-06 10:27:00,177 INFO  [main] index.IndexCollection (IndexCol

## Read the queries

In [140]:
df_queries = pd.read_csv('../data/DBpedia-Entity/collection/v2/queries-v2_stopped.txt',sep='\t',header=None)


## parse queries to the right format for Anserini

In [141]:
df_queries_parsed = df_queries[1]
df_queries_parsed.to_csv('../data/queries.tsv', header = None, sep = '\t')

## query wikipedia by Anserini based on bm25 for 1000 hits

In [144]:
!python -m pyserini.search.lucene \
  --index  ../data/index  \
  --topics ../data/queries.tsv \
  --output ../data/run_anserini.sample.txt \
  --bm25 \
  --hits 1000

Running ../data/queries.tsv topics, saving to ../data/run_anserini.sample.txt...
100%|█████████████████████████████████████████| 467/467 [01:07<00:00,  6.89it/s]


In [145]:
df_query_results = pd.read_csv('../data/run_anserini.sample.txt',sep='\s',header = None)

/tmp/ipykernel_2195342/3548555071.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_query_results = pd.read_csv('../data/run_anserini.sample.txt',sep='\s',header = None)


In [146]:
df_query_results

,0,1,2,3,4,5
0,0,Q0,1142326,1,8.916700,Anserini
1,0,Q0,1828331,2,8.914900,Anserini
2,0,Q0,70128883,3,8.870700,Anserini
3,0,Q0,30170055,4,8.851500,Anserini
4,0,Q0,773026,5,8.737000,Anserini
...,...,...,...,...,...,...
466101,466,Q0,30056920,996,8.498600,Anserini
466102,466,Q0,47029698,997,8.498599,Anserini
466103,466,Q0,42764477,998,8.496600,Anserini
466104,466,Q0,10569752,999,8.495800,Anserini


In [147]:
df_query_results.drop(columns=[1,5],inplace=True)
df_query_results.rename(columns={
    0:'query_index',
    2: 'id',
    3:'hit',
    4 :'score'
},inplace=True)

In [ ]:
## merge the queries and original wikipedia 

In [152]:
df_queries.reset_index(inplace = True)
df_queries.rename(columns={'index':'query_index',
                    0:'query_id',1:'query_string'},inplace=True)
df_query_results = df_query_results.merge(df_queries,on = 'query_index')
df_wiki_all_2022.id = df_wiki_all_2022.id.astype(int)
df_query_results = df_query_results.merge(df_wiki_all_2022, on ='id')
df_query_results.sort_values(by = ['query_index','hit'],inplace=True)

In [158]:
df_query_results

,query_index,id,hit,score,query_id,query_string,url,title,text
0,0,1142326,1,8.916700,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/The%20Green%20Be...,The Green Berets (film),The Green Berets is a 1968 American war film d...
1,0,1828331,2,8.914900,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Hearts%20and%20M...,Hearts and Minds (film),Hearts and Minds is a 1974 American documentar...
3,0,70128883,3,8.870700,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/The%20Losers%20%...,The Losers (1970 film),The Losers released on video as Nam's Angels i...
4,0,30170055,4,8.851500,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Coordinates%20of...,Coordinates of Death,Coordinates of Death (alternative title Target...
5,0,773026,5,8.737000,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Adrian%20Cronauer,Adrian Cronauer,"Adrian Joseph Cronauer (September 8, 1938 – Ju..."
...,...,...,...,...,...,...,...,...,...
466101,466,30056920,996,8.498600,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/River%20Laggan,River Laggan,The River Laggan is a small river on the Scott...
466102,466,47029698,997,8.498599,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/Bowmore%20Sandst...,Bowmore Sandstone Group,The Bowmore Sandstone Group is a sequence of m...
466103,466,42764477,998,8.496600,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/1816%20in%20Scot...,1816 in Scotland,Events from the year 1816 in Scotland.\n\nIncu...
466104,466,10569752,999,8.495800,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/Something%20Special,Something Special,"Something Special may refer to:\n\n ""Something..."


## convert dpedia to wikipedia using https://github.com/TREMA-UNH/DBpediaV2-entity-CAR

In [159]:
df_query_dp_to_wiki = pd.read_csv('/home/parham/entity-ranking/data/DBpediaV2-entity-CAR/qrels/train.pages.cbor-article.entity.graded.qrels'
    ,sep = '\s', header=None)

/tmp/ipykernel_2195342/2452462086.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_query_dp_to_wiki = pd.read_csv('/home/parham/entity-ranking/data/DBpediaV2-entity-CAR/qrels/train.pages.cbor-article.entity.graded.qrels'


In [160]:
df_query_dp_to_wiki[2] = df_query_dp_to_wiki[2].str.split('enwiki:').apply(lambda x:urllib.parse.unquote(x[1]))
df_query_dp_to_wiki.drop(columns=1,inplace=True)
df_query_dp_to_wiki.rename(columns={
    0:'query_id',
    2:'title',
    3:'qrel'
},inplace=True)

In [163]:
df_query_dp_to_wiki.sort_values('query_id')

,query_id,title,qrel
23979,INEX_LD-2009022,Italian cuisine,0
107,INEX_LD-2009022,Black cardamom,0
42745,INEX_LD-2009022,Thai cuisine,0
2923,INEX_LD-2009022,Side dish,0
582,INEX_LD-2009022,Global cuisine,0
...,...,...,...
40483,TREC_Entity-9,Piano trio,0
43810,TREC_Entity-9,Augustus Saint-Gaudens,0
16977,TREC_Entity-9,Académie des Beaux-Arts,1
4566,TREC_Entity-9,David Baker (composer),0


## Get the merger of queries and the qrel scores

In [178]:
df_query_results_merge = df_query_results.merge(df_query_dp_to_wiki,on = ['query_id','title'],how = 'left' )

In [187]:
df_query_results_merge.to_parquet('../data/processed/df_query_results_merge.parquet')

In [2]:
df_query_results_merge = pd.read_parquet('../data/processed/df_query_results_merge.parquet')

In [184]:
# df_query_dp_to_wiki_text = df_query_dp_to_wiki.merge(df_wiki_all_2022, on = 'title', how = 'left')

In [3]:

# df_query_results_merge

In [4]:
## the queries where there is no match, the page has changed from past. 
# df_query_dp_to_wiki_text[df_query_dp_to_wiki_text.id.isna()]

In [5]:
# 4102/48960

In [6]:
# df_diff = pd.concat([df_query_results_merge[df_query_dp_to_wiki_text.columns],df_query_dp_to_wiki_text]).drop_duplicates(keep=False)


In [206]:
df_query_results_merge

,query_index,id,hit,score,query_id,query_string,url,title,text,qrel
0,0,1142326,1,8.916700,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/The%20Green%20Be...,The Green Berets (film),The Green Berets is a 1968 American war film d...,2.0
1,0,1828331,2,8.914900,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Hearts%20and%20M...,Hearts and Minds (film),Hearts and Minds is a 1974 American documentar...,0.0
2,0,70128883,3,8.870700,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/The%20Losers%20%...,The Losers (1970 film),The Losers released on video as Nam's Angels i...,NaN
3,0,30170055,4,8.851500,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Coordinates%20of...,Coordinates of Death,Coordinates of Death (alternative title Target...,2.0
4,0,773026,5,8.737000,INEX_LD-20120111,vietnam war movie,https://en.wikipedia.org/wiki/Adrian%20Cronauer,Adrian Cronauer,"Adrian Joseph Cronauer (September 8, 1938 – Ju...",NaN
...,...,...,...,...,...,...,...,...,...,...
466101,466,30056920,996,8.498600,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/River%20Laggan,River Laggan,The River Laggan is a small river on the Scott...,NaN
466102,466,47029698,997,8.498599,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/Bowmore%20Sandst...,Bowmore Sandstone Group,The Bowmore Sandstone Group is a sequence of m...,NaN
466103,466,42764477,998,8.496600,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/1816%20in%20Scot...,1816 in Scotland,Events from the year 1816 in Scotland.\n\nIncu...,NaN
466104,466,10569752,999,8.495800,TREC_Entity-20,Scotch whisky distilleries on the island of Islay,https://en.wikipedia.org/wiki/Something%20Special,Something Special,"Something Special may refer to:\n\n ""Something...",NaN


In [7]:
# df_query_dp_to_wiki_text.id = df_query_dp_to_wiki_text.id.astype(int)